In [1]:
# https://k3no.medium.com/how-to-query-in-graphql-6ebb3f7085dc
# https://github.com/sushiswap/sushiswap-subgraph

In [82]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import pandas as pd
import dataframe_image as dfi
import os

quickswap = 'https://api.thegraph.com/subgraphs/name/sameepsi/quickswap02'
quickswap_daily = 'https://api.thegraph.com/subgraphs/name/moonster-bsc/quickswap-daypairdataslim'

## Quickswap

In [3]:
sample_transport=RequestsHTTPTransport(
    url=quickswap,
    verify=True,
    retries=3,
)

client = Client(transport=sample_transport)

In [4]:
query = gql('''
query {
  uniswapFactories {
    pairCount
  }
}
''')

response = client.execute(query)
number_of_pairs = int(response['uniswapFactories'][0]['pairCount'])
print(number_of_pairs)

5485


In [5]:
df = pd.DataFrame([])
response_dict = {}
response_list = []


for i in range(number_of_pairs):
    if i % 100 == 0:
        querystring = '''
              query {
              pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                      token0 {symbol}
                      token1 {symbol}
                      id
                  }
                }
                '''
        try:
            response = client.execute(gql(querystring))
            df = df.append(pd.DataFrame.from_dict(response['pairs']))
    #         for row in response['pairs']:
    #             response_list.append(row['token0']['symbol'] + '-' + row['token1']['symbol'])
            print(len(df))
        except:
            pass

df

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3422
3422
3422
3422
3422
3422
3422
3422
3422
3422
3422
3422
3422
3422
3422
3422
3422


,id,token0,token1
0,0x0024739eb63fb6697e63698c93c77b9508f15ab2,{'symbol': 'USDC'},{'symbol': 'EDG'}
1,0x002808d3862cffade1e12bcae7c577f3499cb07a,{'symbol': 'WMATIC'},{'symbol': 'GLCH'}
2,0x004bc35a30d095cabecb3b79ee5833a5b8d68957,{'symbol': 'WMATIC'},{'symbol': 'Light Network'}
3,0x0073f4587a2bc0dfedeb0ed88c38b91e42736588,{'symbol': 'WMATIC'},{'symbol': 'Tel'}
4,0x0079c2fd75239b031c09a4286b72b6d127c3bda8,{'symbol': 'WMATIC'},{'symbol': 'FIRE'}
...,...,...,...
17,0xffa4c743a6d76395256d58bd3cbda61325858cea,{'symbol': 'Safematic'},{'symbol': 'WMATIC'}
18,0xffbbe3e43fc51cfef5250797013e1a1e5d786cc0,{'symbol': 'USDC'},{'symbol': 'BzB'}
19,0xffcbe548f71a6323f557df738e75e9984868395b,{'symbol': 'WMATIC'},{'symbol': 'GFI'}
20,0xffd7c073d0d64e69666111a0f95fa3ac3e857ee4,{'symbol': 'VGTG'},{'symbol': 'QUICK'}


In [6]:
df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                           .replace("}",'', regex= True)\
                                           .replace("'",'', regex= True)
df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                           .replace("}",'', regex= True)\
                                           .replace("'",'', regex= True)
df['Pair'] = df['token0'] + '-' + df['token1']
df.columns = ['id','Token 0', 'Token 1', 'Pair']
df = df[['id','Pair', 'Token 0', 'Token 1']]
df.head(2)

,id,Pair,Token 0,Token 1
0,0x0024739eb63fb6697e63698c93c77b9508f15ab2,USDC-EDG,USDC,EDG
1,0x002808d3862cffade1e12bcae7c577f3499cb07a,WMATIC-GLCH,WMATIC,GLCH


In [88]:
# interesting_tokens = ['YFI', 'WOOFY', 'CRV', 'ILV', 'WBTC', 'WMATIC', 'MATIC', 'AAVE', 'WETH', 'ETH', 'SUSHI', 'UNI', 'SNX', 'LINK', 'DAI', 'USDC', 'USDT', 'FRAX']
# interesting_tokens = ['USDC', 'USDT', 'DAI', 'FRAX', 'IRON']
interesting_tokens = ['MATIC', 'DAI', 'USDC', 'USDT', 'WBTC', 'WETH', 'FRAX', 'WMATIC']

criteria = df['Token 0'].isin(interesting_tokens) & \
           df['Token 1'].isin(interesting_tokens)

df_filtered = df.loc[criteria,:]
df_filtered.head()

,id,Pair,Token 0,Token 1
5,0x2cf7252e74036d1da831d11089d326296e64a728,USDC-USDT,USDC,USDT
64,0x4a35582a710e1f4b2030a3f826da20bfb6703c09,WETH-DAI,WETH,DAI
26,0x4f2cc25028d4d80355c802482463b5b2a93cdb84,WMATIC-WETH,WMATIC,WETH
73,0x527a26525f5483612a1f96cfceacc7032b1c8610,WMATIC-WETH,WMATIC,WETH
73,0x59153f27eefe07e5ece4f9304ebba1da6f53ca88,DAI-USDT,DAI,USDT


In [89]:
df4 = pd.DataFrame([])
lenid = len(df_filtered['id'])

sample_transport=RequestsHTTPTransport(
    url=quickswap_daily,
    verify=True,
    retries=3,
)

client = Client(transport=sample_transport)

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
              query {
  pairDayDatas(where: {pairAddress:"''' + id + '''"} orderBy: date, orderDirection: desc first: 7) {
    date
    dailyVolumeUSD
    reserveUSD
  }
}'''
    response = client.execute(gql(querystring))
    response
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df4 = df4.append(df1)

 1 / 22 pair: 0x2cf7252e74036d1da831d11089d326296e64a728


KeyboardInterrupt: 

In [ ]:
df5 = df4.merge(df_filtered,
                left_on = 'id',
                right_on = 'id',
                how = 'left',
                validate = 'many_to_one')
df5[['dailyVolumeUSD','reserveUSD']] = df5[['dailyVolumeUSD','reserveUSD']].astype(float)
df5['date'] = pd.to_datetime(df5['date'], unit='s')
df5['fee'] = round(df5['dailyVolumeUSD'] * 0.003,6)
df5['1y APR for 100 invested'] = round((100/df5['reserveUSD']) * df5['fee'] * 365,3)
df5['Pair'] = '[Quickswap] ' + df5['Pair']
df5.style.format({'reserveUSD': "{:0<4,.10f}"})

In [87]:
criteria1 = df5['reserveUSD'] > 1
criteria2 = df5['date'] == df5['date'].max()

latest_results = df5.loc[(criteria1 & criteria2), :].sort_values(by = 'date',
                                 ascending = False)\
                    .drop_duplicates(subset = ['Pair'])\
                    .sort_values(by = '1y APR for 100 invested',
                                 ascending = False)\
                    .reset_index(drop = True)\
                    .drop(columns = ['id', 'Token 0', 'Token 1'])\
                    .reset_index(drop = True)[['date', 'Pair', '1y APR for 100 invested',  'reserveUSD', 'dailyVolumeUSD', 'fee']]

df_styled  = latest_results.style.format({'reserveUSD': "{:0<4,.2f}",
                             'dailyVolumeUSD': "{:0<4,.2f}",
                             'fee': "{:0<4,.2f}",
                             '1y APR for 100 invested': "{:0<4,.2f}"})\
              .set_properties(subset=["Pair"], **{'text-align': 'left'})

df_styled

,date,Pair,1y APR for 100 invested,reserveUSD,dailyVolumeUSD,fee
0,2021-06-19 00:00:00,[Quickswap] WMATIC-DAI,72.30,"284,742.00","188,003.04",564.01
1,2021-06-19 00:00:00,[Quickswap] WMATIC-USDT,43.59,"1,109,752.34","441,790.92","1,325.37"
2,2021-06-19 00:00:00,[Quickswap] WMATIC-USDC,20.16,"82,994,472.28","15,278,622.00","45,835.87"
3,2021-06-19 00:00:00,[Quickswap] SUSHI-WMATIC,19.48,184.33,32.80,0.10
4,2021-06-19 00:00:00,[Quickswap] WETH-YFI,8.66,"5,618.64",444.19,1.33
5,2021-06-19 00:00:00,[Quickswap] USDC-AAVE,7.92,"1,502.41",108.64,0.33
6,2021-06-19 00:00:00,[Quickswap] WMATIC-WBTC,7.26,"338,351.97","22,446.41",67.34
7,2021-06-19 00:00:00,[Quickswap] USDC-WETH,6.91,"138,642,200.97","8,743,929.49","26,231.79"
8,2021-06-19 00:00:00,[Quickswap] WBTC-USDC,6.81,"22,987,792.21","1,429,930.01","4,289.79"
9,2021-06-19 00:00:00,[Quickswap] WMATIC-WETH,5.85,"83,035,125.97","4,435,663.69","13,306.99"


In [83]:
dfi.export(df_styled, 'df_styled.png')
os.startfile('df_styled.png')

In [90]:

# ['WFTM', 'USDC', 'DAI', 'FUSD', 'ETH', 'fUSDT', 'FETH']


# hyperswap fantom   https://ftm-info.hyperjump.fi/pairs
# honeyswap polygon  https://info.honeyswap.org/#/pairs